In [1]:
# Testing qbSoftHand

In [3]:
import rospy
import actionlib
from control_msgs.msg import FollowJointTrajectoryAction, FollowJointTrajectoryGoal
from trajectory_msgs.msg import JointTrajectoryPoint
import time


class SofthandGripper:
    def __init__(self, device_id = 1, **kwargs):
        """ QbRobotics SoftHand gripper
        device_id: int: Index of device, as set in its firmware. To determine this value, open the qbGUI controller using root privileges (sudo).
        You can get the qbGui controller at: https://qbrobotics.com/download/qb-softhand-research-downloads/
        After connecting to the device and activating it using qbGUI (top left side of windows), the device ID will be displayed.

        TODO check if device_id should be empty (e.g. '' instead of '0') if device_id is set to 0 in qbgui. This can be checked by 'rostopic list | grep qbhand'
        """
        #super().__init__()

        if not rospy.get_node_uri():
            rospy.init_node("qb_hand_client", anonymous = True)

        self.Name = 'softhand'

        device_id = str(device_id)
        self._topic = f'/qbhand{device_id}/control/qbhand{device_id}_synergy_trajectory_controller/follow_joint_trajectory'

        # Time to complete movement.
        self.motion_duration = 0.7
        self.sleep_duration = 1.2*self.motion_duration

        self._client = actionlib.SimpleActionClient(self._topic, FollowJointTrajectoryAction)
        self.actionGoal = FollowJointTrajectoryGoal()
        self.actionGoal.trajectory.joint_names = [f'qbhand{device_id}_synergy_joint']

        self.actionPoint = JointTrajectoryPoint()
        self.actionPoint.time_from_start = rospy.Duration(self.motion_duration) # seconds

    def open(self, command = 1, sleep = False, wait_for_result = False, **kwargs):
        """ If input command == 1, then  it opens to the max. if 0, it will close. if in between...
        Args :
        command 
        sleep - if true, sleep for the required time to open/close gripper
        wait_for_result: wait for result from the action server""" 

        assert (0 <= command <= 1)
        # Command must be between 0 and 1. By default it is one

        actualcommand = 1-command  # If actualcommand is 0, gripper will open to the max. If 1, it will close. (Thats how softhand actions work).

        self.actionPoint.positions = [actualcommand] 
        self.sendActionGoal(wait_for_result)
        if sleep:time.sleep(self.sleep_duration)

    def close(self, command = 1, sleep = False, wait_for_result = False, **kwargs):
        """Args :
        command 
        sleep - if true, sleep for the required time to open/close gripper
        wait_for_result: wait for result from the action server"""
        assert (0 <= command <= 1)

        actualcommand = command

        self.actionPoint.positions = [actualcommand] 
        self.sendActionGoal(wait_for_result)
        if sleep:time.sleep(self.sleep_duration)

    def sendActionGoal(self, wait_for_result = False):

        self.actionGoal.trajectory.points = [self.actionPoint]

        #rospy.loginfo("Starting sending softhand goal...")
        try:
            rospy.loginfo("{} goal sent: {}".format(self.Name, str(self.actionPoint.positions)))
            self._client.send_goal(self.actionGoal)
            if wait_for_result:
                res = self._client.get_result(self._topic)
                while res == None:
                    res = self._client.get_result(self._topic)
                    time.sleep(0.2)
                rospy.loginfo("{} result: {}".format(self.Name, res))

            self._state = 0

        except Exception as e:
            # Since a state failure not necessarily causes a behavior failure, it is recommended to only print warnings, not errors.
			# Using a linebreak before appending the error log enables the operator to collapse details in the GUI.
            rospy.loginfo('Failed to send the goal command:\n{}'.format(str(e)))
            self._error = True

    def grasp(self, **kwargs):
        self.close()


if __name__ == '__main__':
    gripper_client = SofthandGripper(device_id = 1)
    gripper_client.open()
    time.sleep(4)
    gripper_client.close(command = 0.7)
    time.sleep(4)
    gripper_client.open()

[INFO] [1744635448.339111]: softhand goal sent: [0]
[INFO] [1744635452.343070]: softhand goal sent: [0.7]
[INFO] [1744635456.347142]: softhand goal sent: [0]


In [2]:
gripper_client = SofthandGripper(device_id = 1)
gripper_client.open()
time.sleep(4)
gripper_client.close(command = 0.7)
time.sleep(4)
gripper_client.open()

[INFO] [1744634799.440484]: softhand goal sent: [0]
[INFO] [1744634803.443165]: softhand goal sent: [0.7]
[INFO] [1744634807.447426]: softhand goal sent: [0]
